EXTRACT LINKS  

In [ ]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By



service = Service(r"C:\Users\iheba\OneDrive\Bureau\chromedriver.exe")
driver = webdriver.Chrome(service=service)

# Login to LinkedIn



driver.get('https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiWuuGn5dWAAxWfV6QEHXjUAZoQFnoECBwQAQ&url=https%3A%2F%2Fwww.linkedin.com%2Fjobs%2Fmechanical-technician-jobs&usg=AOvVaw1snUN_sFUG5DRJGxFzfJRk&opi=89978449')

# Scroll down until the bottom of the page or the button is found
last_height = driver.execute_script('return document.documentElement.clientHeight')

while True:
    driver.execute_script('window.scrollTo(0, document.documentElement.scrollHeight);')
    try : 
      if driver.find_element(By.XPATH,'//*[@id="main-content"]/section[2]/button') : 
        button= WebDriverWait(driver, 2).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="main-content"]/section[2]/button')))
        button.click()
        time.sleep(3)
      else : 
          break 
    except Exception : 
        pass 
    time.sleep(2)

    new_height = driver.execute_script('return document.documentElement.scrollHeight')
    time.sleep(1)
    # Try to find the button element by its id, class or text
    
    if new_height == last_height :
        break
    last_height = new_height
# Click the button if it exists




In [ ]:
import  time 
import  requests
from bs4 import BeautifulSoup
import re
links=[]
    
soup = BeautifulSoup(driver.page_source, 'html.parser')

while True  :  
   
    job_postings = soup.find_all('div',class_='base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card')
    for job_posting in job_postings : 
        link = job_posting.find('a', class_='base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]').get("href")
        print(link)
        links.append(link)
    time.sleep(5)

SAVED INTO CSV FORMAT


In [ ]:
with open('linkedin_links', 'w') as f:
        for item in links:
            f.write(item + '\n')


In [ ]:
print(len(links))

In [ ]:
links=[]
with open('linkedin_links', 'r') as file:
    # Read each line of the file and append it to the 'links' list
    for line in file:
        links.append(line.strip()) 
print(links)
print(len(links))

EXTRACT INFORMATION FROM LIKNS 

In [ ]:
import csv
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import  time 
import  requests
from bs4 import BeautifulSoup
import re

import pandas as pd

# Set up Selenium WebDriver (e.g., Chrome)
service = Service(r"C:\Users\iheba\OneDrive\Bureau\chromedriver.exe")
driver = webdriver.Chrome(service=service)

# Login to LinkedIn
driver.get('https://www.linkedin.com/login')
driver.find_element(By.ID, 'username').send_keys('ihebakerr@gmail.com')
driver.find_element(By.ID, 'password').send_keys('nTtiQ.zTV5Qr6!K')
driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button').click()

# You might need to add an appropriate wait here to ensure the login is successful.
# For example, WebDriverWait(driver, 30).until(EC.title_contains("LinkedIn"))
data=[]
with open(f'linkedin_job_results.csv', 'w', newline='', encoding='utf-8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Job Title', 'Company', 'Location', 'Job Link', "Description"])
# Loop through the list of job links
  for url in links:
    # Open the web page
    driver.get(url)

    # Find and click the "See more" button using Selenium
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.jobs-description__footer-button'))
        )
        time.sleep(5)
        driver.find_element(By.CSS_SELECTOR, 'button.jobs-description__footer-button').click()
    except Exception:
        pass
    time.sleep(5)
    # Wait for the new content to load
    # Add appropriate wait conditions if needed
    response = requests.get(url, verify=True)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    job_postings = soup.find_all('div', {'class': 'grid__col grid__col--xl-17 mt4'})
    
    
    job_title=""
    company_name=""
    location=""
    detail=""

# Extract relevant information from each job posting and store it in a list of dictionaries
    for job_posting in job_postings:
        try : 
            if job_posting.find('h1', class_='t-24 t-bold jobs-unified-top-card__job-title') : 
               job_title=job_posting.find('h1', class_='t-24 t-bold jobs-unified-top-card__job-title').text.strip().replace(",", "")
               print("the title is : " + job_title)
            else :
               job_title = None 
               print(None)
                 
         
            if job_posting.find('div', class_='jobs-unified-top-card__primary-description'): 
               detailespace =job_posting.find('div', class_='jobs-unified-top-card__primary-description').text.strip().replace(",", "")
               company_name_status = re.sub(re.compile(r"\s+") , " ", detailespace)
               words = company_name_status.split(' ')
               company_name = words[0].replace(",", " ")
               location = words[2].replace(",", "")
               print(location)
               print("the  company name :" + company_name)
            else  : 
                company_name = None 
                location=None
                print(None)
                
        
    
               
        
         
            if     job_posting.find('div', id='job-details') : 
              
               detailespace =job_posting.find('div', id='job-details').text.strip().replace(",", "")
               detail = re.sub(re.compile(r"\s+") , " ", detailespace)
               print("the description :" + detail)
            else : 
                detail= None
                print(None)
                
       
               
             
         
            writer.writerow([job_title, company_name,location,url, detail])
        except Exception : 
            pass
     



     







In [ ]:
import pandas as pd

df=pd.read_csv('linkedin_job_results.csv')
df